In [1]:
import readfof
from pyspark.sql import SparkSession
import numpy as np
import scipy.spatial as SS


In [2]:
spark = SparkSession.builder \
        .master("spark://master:7077")\
        .appName("CosmoSparkApplication")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/30 10:10:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

In [4]:
# Read data
def read_cosmo_data(file_path):

    # Read Fof
    FoF = readfof.FoF_catalog(
        file_path,           # simulation directory
        2,                  # snapnum, indicating the redshift (z=1)
        long_ids = False,
        swap = False,
        SFR = False,
        read_IDs = False
        )

    return FoF

In [5]:
# get masses and positions

def get_pos_mass(FoF):

    pos = FoF.GroupPos/1e06             # Halo positions in Gpc/h 
    mass_raw = FoF.GroupMass * 1e10     # Halo masses in Msun/h

    dim = pos.shape[0]
    pos_mass_matrix = np.hstack([pos, mass_raw.reshape(dim, 1)])

    return pos_mass_matrix

In [6]:
# mass cut function
def mass_filter(pos_mass_element, cut):

    mass = pos_mass_element[3]
    pos = pos_mass_element[0:3]
    if mass >= cut:
        return pos_mass_element


In [7]:
# simulations parameter

sim_pars_file = np.loadtxt("/mnt/cosmo_GNN/latin_hypercube_params.txt", dtype=float)

In [8]:
path_list = "/mnt/cosmo_GNN/Data/" + str(13)

In [9]:
test_FoF = read_cosmo_data(path_list)

In [10]:
pos_mass_array = get_pos_mass(test_FoF)

In [11]:
cut = np.quantile(pos_mass_array[:, 3], 0.997)

In [12]:
pos_mass_rdd = sc.parallelize(pos_mass_array)\
                .map(lambda el: mass_filter(el, cut))

In [24]:
cosa = pos_mass_rdd.take(3)[0]
cosa[0:3]


array([0.74910957, 0.39241824, 0.7223062 ], dtype=float32)

In [25]:
# get KDTree
def get_edges_idx(pos_mass):
    pos = pos_mass[0:3]
    kd_tree = SS.KDTree(pos, leafsize=16, boxsize=1.0001)
    edge_idx = kd_tree.query_pairs(r=0.2, output_type="ndarray")

    return edge_idx

In [26]:
edge_idx_rdd = pos_mass_rdd.reduce(get_edges_idx)

24/08/30 10:15:09 WARN TaskSetManager: Lost task 0.0 in stage 10.0 (TID 101) (10.67.22.172 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/rdd.py", line 1922, in func
    yield reduce(f, iterator, initial)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
TypeError: get_edges_idx() takes 1 positional argument but 2 were given

	at org.apache.spark.api.python.Ba

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13 in stage 10.0 failed 4 times, most recent failure: Lost task 13.3 in stage 10.0 (TID 147) (10.67.22.163 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/rdd.py", line 1922, in func
    yield reduce(f, iterator, initial)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/uti

24/08/30 10:15:10 WARN TaskSetManager: Lost task 10.3 in stage 10.0 (TID 153) (10.67.22.65 executor 1): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 13 in stage 10.0 failed 4 times, most recent failure: Lost task 13.3 in stage 10.0 (TID 147) (10.67.22.163 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/rdd.py", line 1922, in func
    yield reduce(f, iterator, initial)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/util.py",

24/08/30 10:15:10 WARN TaskSetManager: Lost task 15.3 in stage 10.0 (TID 159) (10.67.22.240 executor 3): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 13 in stage 10.0 failed 4 times, most recent failure: Lost task 13.3 in stage 10.0 (TID 147) (10.67.22.163 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/rdd.py", line 1922, in func
    yield reduce(f, iterator, initial)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/util.py"